In [121]:
%pip install -q langchain langchain_core pandas streamlit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [110]:
%load_ext autoreload
%autoreload 2
from image_data_extraction import image_analysis
import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import json

# Load environment variables from .env file for secure credential management
load_dotenv()

# Retrieve Azure and OpenAI configuration from environment variables
chat_model = os.getenv("AOAI_CHAT_DEPLOYMENT_NAME", "gpt-4o")
aoai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_key = os.getenv("AZURE_OPENAI_API_KEY")

# Initialize Azure OpenAI chat client for LLM-based summarization
llm_client = AzureChatOpenAI(
    azure_deployment=chat_model,
    api_version="2023-05-15",
    temperature=0.3,
    model_name=chat_model,
    azure_endpoint=aoai_endpoint,
    api_key=aoai_key,
)

# from build_graph import build_graph
image_path = "azure_architecture_medium.png"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
from prompts import data_extraction_prompt, cost_calculation_prompt, json_output_format

result = image_analysis(image_path=image_path, llm_client=llm_client, prompt=data_extraction_prompt)
response = result.content.replace("```json\n", "").replace("\n```", "")
data = json.loads(response)["nodes"] 
cost_calculation_prompt = cost_calculation_prompt.format(
    data=data,
    json_output_format=json_output_format
)    
response = llm_client.invoke(
    [
        HumanMessage(
            content=cost_calculation_prompt,
        )
    ]
)
response_text = response.content.replace("```json\n", "").replace("\n```", "")

In [120]:
import pandas as pd
result_json = json.loads(response_text)
df = pd.DataFrame(result_json)
df.head(len(df))

,service,sku,quantity,unit_price,monthly_cost,currency,assumptions
0,Private DNS Zones,Standard,1,0.5000,0.50,USD,Standard pricing for Private DNS Zones
1,Application Gateway with Azure WAF,WAF_v2,1,250.0000,250.00,USD,Includes WAF_v2 with high availability
2,Azure App Service,PremiumV3 P1v3,3,114.6000,343.80,USD,Three instances for high availability
3,Managed Identity,Free,1,0.0000,0.00,USD,Managed Identity is free of charge
4,Azure Key Vault,Standard,1,0.0300,3.00,USD,"Assumes 100,000 operations per month"
5,Azure Storage,Hot Block Blob,100,0.0184,1.84,USD,100 GB of hot storage
6,Azure AI Search,Standard S1,1,250.0000,250.00,USD,Standard S1 tier for production workloads
7,Azure Cosmos DB,Provisioned Throughput,400,0.0080,96.00,USD,400 RU/s provisioned throughput
8,Azure AI Foundry Account,Standard,1,100.0000,100.00,USD,Standard pricing for AI Foundry
9,Azure OpenAI Model,Standard,1,1000.0000,1000.00,USD,Standard pricing for OpenAI model usage
